강화학습을 구성하는 여러 요소 중 3가지는 아래와 같다.

1. 액션 의존성- Agent가 하는 action에 따라 받는 reward가 다르다.

2. 시간 의존성- action에 대한 reward를 바로바로 받는 것이 아니다. 얼마의 시간이 경과한 뒤 reward를 받을 수도 있고 sparse하다.

3. 상태 의존성- Environment의 상태에 따라 받는 reward가 다르다.

이번 예시에서는 bandit을 당기면 그에 따라 다른 확률로 보상이나 패널티를 받는다. 2, 3번을 고려하지 않는다.

아래는 강화학습 과정을 나타낸다.

1. Agent는 Environment로부터 받은 상태를 기반으로 action을 생산한다. (단, 이번 Bandit 예제에서는 Environement의 상태는 없다.)

2. Environment는 Agent로부터 받은 action을 기반으로 reward를 생산한다. 그리고 자신을 그 action을 기반으로 상태를 업데이트 한다. (단, Environment의 상태가 없기 때문에 생략한다.)

3. Agent는 앞서 생산했던 action을 주고 어떤 reward를 받았는지를 통해 학습을 진행한다.

In [224]:
import numpy as np

class Environment:
    def __init__(self, bandit_arms):
        self.bandit_arms = list(bandit_arms)
        self.num_arms = len(bandit_arms)

    def pull_bandit(self, action):
        result = np.random.randn()
        return 1 if result > self.bandit_arms[action] else -1

env = Environment([0.2, 0, -0.2, -2])

In [225]:
import torch

class Agent():
    def __init__(self, num_actions):
        self.weights = torch.ones(num_actions, requires_grad=True)
        self.policy = torch.nn.Softmax(dim=0)
        self.optimizer = torch.optim.Adam([self.weights], lr=1e-3)

    def action(self) -> int:
        with torch.inference_mode():
            output = self.policy(self.weights)
            action = torch.multinomial(output, 1)
        return action.detach().item()

    def train(self, action, reward):
        responsible_output = self.weights[action]
        loss = -torch.log(responsible_output) * reward
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

agent = Agent(env.num_arms)

In [226]:
total_episode = 1000
total_reward = np.zeros(env.num_arms)

for i in range(total_episode):
    action = agent.action()
    reward = env.pull_bandit(action)
    agent.train(action, reward)
    total_reward[action] += reward
    if i % 50 == 49:
        print(i + 1, total_reward)

50 [-3.  2.  0.  9.]
100 [-9.  3.  1. 15.]
150 [-7.  1.  3. 27.]
200 [-6. -2.  4. 38.]
250 [-7. -4.  3. 52.]
300 [-13. -14.  -2.  63.]
350 [-15.  -6.   5.  74.]
400 [-19.  -7.   3.  87.]
450 [-29.   0.   7.  98.]
500 [-31.  -5.  16. 116.]
550 [-32.  -7.  21. 132.]
600 [-31. -10.  23. 148.]
650 [-30.  -5.  31. 158.]
700 [-31.  -9.  29. 171.]
750 [-36. -12.  32. 186.]
800 [-37. -14.  34. 207.]
850 [-35. -19.  36. 224.]
900 [-36. -18.  36. 240.]
950 [-43. -18.  42. 257.]
1000 [-42. -11.  41. 274.]


In [227]:
agent.weights

tensor([0.9146, 0.9718, 1.0808, 1.5027], requires_grad=True)